# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [2]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/new_run_scripts


## Define machine and code 

In [3]:
facility='cori'
img_size=128
run_type='2dgan'
assert facility in ['cori','summit'], "Error%s"%(facility)
assert img_size in [128,512]
assert run_type in ['2dgan','2dcgan','3dgan','3dcgan']

## Read ref_launch.yaml and define dictionary

In [34]:
launch_cfile=start_dir+'/ref_launch.yaml'

with open(launch_cfile) as f:
    config_dict= yaml.load(f, Loader=yaml.SafeLoader)

In [35]:
## Read ref_launch.yaml
dict_pars=config_dict[facility][img_size][run_type]
dict_pars.update({'nodes':1,'job_name':'analysis_'+run_type,
                 'queue':'debug','bins':'unenven','cores':64,'time':'00:30:00'})

print(dict_pars)
staging_loc=dict_pars['staging_loc']

{'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/DDP_new_loss/', 'config': 'config_2dgan.yaml', 'staging_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'nodes': 1, 'job_name': 'analysis_2dgan', 'queue': 'debug', 'bins': 'unenven', 'cores': 64, 'time': '00:30:00'}


In [36]:
%%bash -s "$staging_loc"
ls $1

20210205_062936_cgan_predict_0.65_m2
20210205_063709_cgan_predict_1.1_m2
20210205_120608_cgan_predict_0.5_m2
20210205_122504_cgan_predict_0.8_m2
20210209_075831_lambda2.0
20210408_064020_old_gan_lambda2.0
20210408_161912_train_newloss_noddpcode_works
20210411_063412_ddp_code_notusingddp
20210412_114653_2nodes_4gpus_each
20210412_133952_2nodes_16_bsize32
20210412_150439_2nodes_16_bsize16
20210412_70407_ddp_code_4gpus_single_node
20210414_102623_single_gpu_fmloss_80
20210414_154956_1node_8_fmloss
20210414_182903_cgan_ddp_1node_2
20210414_82846_2nodes_16_fmloss
20210415_140527_gclip1.0
20210415_151529_noclip
20210415_170935_optD_infront
20210415_180935_cgan_optD_infront
20210415_205808_cgan_bs64_60epochs
simple_gan_best_runs


In [37]:
ip_folder='20210415_205808_cgan_bs64_60epochs'

dict_pars['ip_folder']=dict_pars['staging_loc']+ip_folder

In [38]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code'

python $code_dir/post_analysis_pandas.py -f {ip_folder}  -v {val_file} -c {cores} -bin {bins}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [42]:
fname='/../batch_analysis_{0}.sh'.format(run_type)
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/../batch_analysis_2dgan.sh


In [32]:
### Move to staging locations in project space:
# os.chdir(dict_pars['staging_loc'])

In [43]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_2dgan

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code'

python $code_dir/post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq20210415_205808_cgan_bs64_60epochs  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy -c 64 -bin unenven
conda deactivate
echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [34]:
# %%bash -s "$filename" ## Use python variable in bash
# module load cgpu
# sbatch $1

In [35]:
200000*0.25/(16)

3125.0